# Projet de DataMining :  

Dans ce projet nous allons recommander des images en fonctions d'images déjà "liké".
Pour cela nous avons commencé par traiter une base de 733 images en recupérant le maximun de données. On recupére notament les données Exif, la taille de l'image, son orientation, ect ...  
La base de données d'images vient du lien suivant:
https://www.kaggle.com/aksha05/flower-image-dataset
## Requirements : 

In [ ]:
from PIL import Image
from pandas import json_normalize
import pandas as pd
import json
import os
import numpy
import math
import matplotlib.pyplot as plot
import time
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans

## Récupération des données et organisation de ces données dans un fichier json:

## Détection des couleurs prédominantes

In [ ]:
start_time = time.time()
imgfile = Image.open("images/flowers/daisies_00046.jpg")
n_clusters = 4 #int(input("Nombre de clusters pour l'algorithme des K-Means: "))
numarray = numpy.array(imgfile.getdata(), numpy.uint8)
clusters = KMeans(n_clusters)
clusters.fit(numarray)
npbins = numpy.arange(0, n_clusters+1)
histogram = numpy.histogram(clusters.labels_, bins=npbins)
labels = numpy.unique(clusters.labels_)
index1 = {}
for i,element in enumerate(histogram[0]):
    index1[element] = i
#print(index1)
histoSort = sorted(histogram[0], reverse=True)
#print(histoSort)
index2 = {}
for i,element in enumerate(histoSort):
    index2[i] = element
#print(index2)
barlist = plot.bar(labels, histoSort)
colors = []
for k in range(n_clusters):
    i = index1[index2[k]]
    colors.append('#%02x%02x%02x' % (
    math.ceil(clusters.cluster_centers_[i][0]), 
        math.ceil(clusters.cluster_centers_[i][1]),
    math.ceil(clusters.cluster_centers_[i][2])))
    barlist[k].set_color('#%02x%02x%02x' % (
    math.ceil(clusters.cluster_centers_[i][0]), 
        math.ceil(clusters.cluster_centers_[i][1]),
    math.ceil(clusters.cluster_centers_[i][2])))
plot.show()
end_time = time.time()
total_time = end_time - start_time
print(total_time)

In [ ]:
start_time = time.time()
imgfile = Image.open("images/flowers/daisies_00046.jpg")
n_clusters = 4
numarray = numpy.array(imgfile.getdata(), numpy.uint8)
clusters = MiniBatchKMeans(n_clusters)
clusters.fit(numarray)
npbins = numpy.arange(0, n_clusters+1)
histogram = numpy.histogram(clusters.labels_, bins=npbins)
labels = numpy.unique(clusters.labels_)
index1 = {}
for i,element in enumerate(histogram[0]):
    index1[element] = i
#print(index1)
histoSort = sorted(histogram[0], reverse=True)
#print(histoSort)
index2 = {}
for i,element in enumerate(histoSort):
    index2[i] = element
#print(index2)
barlist = plot.bar(labels, histoSort)
colors = []
colorsDec = []
for k in range(n_clusters):
    i = index1[index2[k]]
    colors.append('#%02x%02x%02x' % (
    math.ceil(clusters.cluster_centers_[i][0]), #RGB normalement 0:R 1:G 2:B
        math.ceil(clusters.cluster_centers_[i][1]),
    math.ceil(clusters.cluster_centers_[i][2])))
    
    colorsDec.append([math.ceil(clusters.cluster_centers_[i][0]), #RGB normalement 0:R 1:G 2:B
        math.ceil(clusters.cluster_centers_[i][1]),
    math.ceil(clusters.cluster_centers_[i][2])])
    
    barlist[k].set_color('#%02x%02x%02x' % (
    math.ceil(clusters.cluster_centers_[i][0]), 
        math.ceil(clusters.cluster_centers_[i][1]),
    math.ceil(clusters.cluster_centers_[i][2])))

plot.show()
end_time = time.time()
total_time = end_time - start_time
print(total_time)

On recupere un base de données comprenant un certain nombre de couleurs permetant d'avoir des réferences

In [ ]:
jsondataColor = json.load(open("colours_rgb_shades.json"))
array = []

for data in jsondataColor:
    array.append([data["Color Name"], data["Credits"], data["R;G;B Dec"]])
    
dataframe = pd.DataFrame(array, columns=["Color Name", "Credits", "R;G;B Dec"])
dataframe = dataframe.astype(
    dtype={"Color Name": "<U200", "Credits": "<U200", "R;G;B Dec": "<U200"}
)
selectColor = dataframe.loc[(dataframe["Credits"] == "N,V,X")  | (dataframe["Credits"] == "N,X")]
print(selectColor)



selectColor.to_json (r'dataSelectColor.json',orient = "index",indent = 4)



Ici on remarque bien que l'algorithme MiniBatchKMeans va beaucoup plus vite que l'algorithme des KMeans bien qu'il soit moins précis. Nous décidons de l'utiliser tout de même car nous ne pouvons pas nous permettre de prendre trop de temps pour traiter les 700 images !

Nous obtenons des coulours et maintenant nous essayons d'approximer les couleurs

In [ ]:
with open('dataSelectColor.json') as mon_fichier:
    data = json.load(mon_fichier)
colorList = [] 
for key in data :
    colorList.append([data[key]['Color Name'],data[key]['R;G;B Dec'].split(";")])
#modilfication type RGB 
for element in colorList :
    element[1][0] = int(element[1][0])
    element[1][1] = int(element[1][1])
    element[1][2] = int(element[1][2])

ListeCouleurs = colorList

On utilise les couleurs de réference dans notre detection de couleurs

In [ ]:
print(colorsDec)

data = {}
for i in range(0,4) : 
    R = colorsDec[i][0]
    G = colorsDec[i][1]
    B = colorsDec[i][2]


    #ListeCouleurs = [[255,255,255],[0,0,255],[0,0,125],[0,255,0],[255,0,0],[0,0,0]]
    normeMin = 255
    couleursRef = []
    for couleurs in ListeCouleurs :
        norme = (abs(couleurs[1][0]-R) + abs(couleurs[1][1]-G) + abs(couleurs[1][2]-B))/3
        if norme <= normeMin : 
            couleursRef = couleurs
            normeMin = norme
    data["couleur " + str(i)] = couleursRef
print(data)


On modifie le fichier JSON pour les couleurs 

On passe maintenant à l'application de l'exemple au projet